# Interpretable Clustering with Latent Dirichlet Allocation

In [1]:
import numpy as np
import pandas as pd
import pyLDAvis.gensim
from gensim import corpora
from gensim.models import LdaModel
from sklearn.metrics import normalized_mutual_info_score as NMI
pyLDAvis.enable_notebook()

/Users/taewook/anaconda/lib/python3.5/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In this tutorial,

I will show that the combination of LDA(Latent Dirichlet Allocation) model + pyLDAVis

can be powerful method for interpretable clustering.

## 1. Load Data

In [2]:
data = pd.read_csv('iris.csv')
dummy_data = data.copy()

In [3]:
class_var = 'species'
input_vars = [var for var in data.columns if var != class_var]

In [4]:
data[class_var].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

#### Perform clustering with #cluster = 3

In [5]:
n_cluster = 3

## 2. Discretize Continuous Variables

In [6]:
for var in input_vars:
    dummy_data[var] = pd.cut(data[var], bins=3)

In [7]:
dummy_data.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,"(4.296, 5.5]","(2.8, 3.6]","(0.994, 2.967]","(0.0976, 0.9]",setosa
1,"(4.296, 5.5]","(2.8, 3.6]","(0.994, 2.967]","(0.0976, 0.9]",setosa
2,"(4.296, 5.5]","(2.8, 3.6]","(0.994, 2.967]","(0.0976, 0.9]",setosa


## 3. Preprocess data to be input format of gensim LDA model

#### Convert Each Row to Token List

In [8]:
rows = []
for _, row in dummy_data.iterrows():
    rows.append(['{}_{}'.format(k,v) for k,v in row.items() if k != class_var])

In [9]:
rows[0]

['sepal_length_(4.296, 5.5]',
 'sepal_width_(2.8, 3.6]',
 'petal_length_(0.994, 2.967]',
 'petal_width_(0.0976, 0.9]']

#### Prepare dictionary for values of variables & Convert Structured Data into Corpus Format

In [10]:
dictionary = corpora.Dictionary(rows)
row_var_matrix = [dictionary.doc2bow(row) for row in rows]

## 4. Perform clustering with LDA model & Interpreting Clusters

In [11]:
lda = LdaModel(row_var_matrix, num_topics=n_cluster, id2word = dictionary, passes=50)

In [12]:
clustering_result = pyLDAvis.gensim.prepare(lda, row_var_matrix, dictionary)

#### With PyLdaVis, we can explore each cluster interactively!

In [13]:
clustering_result

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      34.587836        1       1 -0.159395  0.193160
1      34.223469        1       2  0.327624 -0.003441
2      31.188695        1       3 -0.168229 -0.189719, topic_info=     Category       Freq                         Term      Total  loglift  \
term                                                                        
6     Default  53.000000  petal_length_(2.967, 4.933]  53.000000  12.0000   
10    Default  53.000000       petal_width_(0.9, 1.7]  53.000000  11.0000   
5     Default  46.000000       petal_width_(1.7, 2.5]  46.000000  10.0000   
7     Default  46.000000    petal_length_(4.933, 6.9]  46.000000   9.0000   
1     Default  49.000000    petal_width_(0.0976, 0.9]  49.000000   8.0000   
3     Default  49.000000  petal_length_(0.994, 2.967]  49.000000   7.0000   
9     Default  58.000000    sepal_length_(4.296, 5.5]  58.000000   6.0000   
11    Default  70.000000      sepal_length_(5.5, 6.7]  70.000000   5.0000   
0     Default  20.000000      sepal_length_(6.7, 7.9]  20.000000   4.0000   
4     Default  46.000000     sepal_width_(1.998, 2.8]  46.000000   3.0000   
8     Default  15.000000       sepal_width_(3.6, 4.4]  15.000000   2.0000   
2     Default  87.000000       sepal_width_(2.8, 3.6]  87.000000   1.0000   
6      Topic1  52.563559  petal_length_(2.967, 4.933]  53.231590   1.0490   
10     Topic1  52.562687       petal_width_(0.9, 1.7]  53.231615   1.0490   
4      Topic1  32.955455     sepal_width_(1.998, 2.8]  46.828154   0.7103   
11     Topic1  41.940798      sepal_length_(5.5, 6.7]  70.460677   0.5429   
2      Topic1  20.201463       sepal_width_(2.8, 3.6]  87.440063  -0.4035   
9      Topic1   5.332542    sepal_length_(4.296, 5.5]  58.734177  -1.3375   
8      Topic1   0.325571       sepal_width_(3.6, 4.4]  15.680734  -2.8129   
0      Topic1   0.337059      sepal_length_(6.7, 7.9]  20.890171  -3.0651   
7      Topic1   0.330116    petal_length_(4.933, 6.9]  46.772868  -3.8919   
5      Topic1   0.329821       petal_width_(1.7, 2.5]  46.772876  -3.8928   
3      Topic1   0.323969  petal_length_(0.994, 2.967]  49.978541  -3.9770   
1      Topic1   0.323968    petal_width_(0.0976, 0.9]  49.978540  -3.9770   
1      Topic2  49.322085    petal_width_(0.0976, 0.9]  49.978540   1.0590   
3      Topic2  49.322085  petal_length_(0.994, 2.967]  49.978541   1.0590   
8      Topic2  14.991231       sepal_width_(3.6, 4.4]  15.680734   1.0273   
9      Topic2  53.068650    sepal_length_(4.296, 5.5]  58.734177   0.9708   
2      Topic2  36.320594       sepal_width_(2.8, 3.6]  87.440063   0.1937   
0      Topic2   0.328480      sepal_length_(6.7, 7.9]  20.890171  -3.0803   
4      Topic2   0.334140     sepal_width_(1.998, 2.8]  46.828154  -3.8704   
7      Topic2   0.327784    petal_length_(4.933, 6.9]  46.772868  -3.8884   
5      Topic2   0.327772       petal_width_(1.7, 2.5]  46.772876  -3.8885   
10     Topic2   0.328863       petal_width_(0.9, 1.7]  53.231615  -4.0145   
6      Topic2   0.328845  petal_length_(2.967, 4.933]  53.231590  -4.0146   
11     Topic2   0.340296      sepal_length_(5.5, 6.7]  70.460677  -4.2607   
5      Topic3  46.115283       petal_width_(1.7, 2.5]  46.772876   1.1510   
7      Topic3  46.114968    petal_length_(4.933, 6.9]  46.772868   1.1509   
0      Topic3  20.224633      sepal_length_(6.7, 7.9]  20.890171   1.1327   
11     Topic3  28.179584      sepal_length_(5.5, 6.7]  70.460677   0.2487   
2      Topic3  30.918006       sepal_width_(2.8, 3.6]  87.440063   0.1255   
4      Topic3  13.538558     sepal_width_(1.998, 2.8]  46.828154  -0.0758   
8      Topic3   0.363932       sepal_width_(3.6, 4.4]  15.680734  -2.5981   
1      Topic3   0.332488    petal_width_(0.0976, 0.9]  49.978540  -3.8476   
3      Topic3   0.332487  petal_length_(0.994, 2.967]  49.978541  -3.8476   
10     Topic3   0.340066       petal_width_(0.9, 1.7]  53.231615  -3

#### To enhance interpretability, merge adjacent intervals

In [14]:
def merge_adjacent_intervals(intervals_for_var):
    
    def get_begin_and_end_points(interval):
        return [float(x) for x in interval.split('_(')[1].replace(']','').split(', ')]
    
    var_name = intervals_for_var[0].split('_(')[0]
    intervals_for_var = sorted([get_begin_and_end_points(i) for i in intervals_for_var])
    
    prev_num_intervals = len(intervals_for_var)
    
    while(True):
        
        for idx, interval in enumerate(intervals_for_var):
            if idx < len(intervals_for_var) - 1:
                curr_interval = intervals_for_var[idx]
                next_interval = intervals_for_var[idx+1]
                if curr_interval[1] == next_interval[0]:
                    merged_interval = [curr_interval[0], next_interval[1]]
                    intervals_for_var[idx+1] = merged_interval
                    intervals_for_var.remove(curr_interval)
                
        curr_num_intervals = len(intervals_for_var)
        if prev_num_intervals > curr_num_intervals:
            prev_num_intervals = curr_num_intervals
        else:
            break
    
    intervals_for_var = ['{}_({}, {}]'.format(var_name, i[0], i[1]) for i in intervals_for_var]
    return intervals_for_var

def postprocess_intervals(var_names, intervals):
    intervals_by_var = []
    for var in var_names:
        intervals_for_var = [x for x in intervals if var == x[:len(var)]]
        if len(intervals_for_var) >= 2:
            intervals_for_var = merge_adjacent_intervals(intervals_for_var)
        intervals_by_var.append(intervals_for_var)
    return intervals_by_var

#### Now, we can describe clusters in terms of intervals of variables

In [15]:
cutoff = 0.1 # P(value of variable | cluster) (Same as P(word | topic) in LDA Model)
for i in range(n_cluster):
    variables = postprocess_intervals(input_vars,
                                       [dictionary.id2token[x[0]] for x in lda.get_topic_terms(i) if x[1] > cutoff])
    print('{} th cluster'.format(i), variables)
    print('')

0 th cluster [['sepal_length_(5.5, 6.7]'], ['sepal_width_(1.998, 2.8]'], ['petal_length_(2.967, 4.933]'], ['petal_width_(0.9, 1.7]']]

1 th cluster [['sepal_length_(4.296, 5.5]'], ['sepal_width_(2.8, 3.6]'], ['petal_length_(0.994, 2.967]'], ['petal_width_(0.0976, 0.9]']]

2 th cluster [['sepal_length_(5.5, 7.9]'], ['sepal_width_(2.8, 3.6]'], ['petal_length_(4.933, 6.9]'], ['petal_width_(1.7, 2.5]']]



## 5. Clustering performance evaluation

#### Get cluster(=topic) label of each row

In [16]:
def get_cluster_label(lda, row):
    return np.argmax([x[1] for x in lda.get_document_topics(row)])

In [17]:
cluster_labels = [get_cluster_label(lda, row) for row in row_var_matrix]
NMI(data.species, cluster_labels)

0.8243868199386184

#### Compare NMI with K-means & Agglomerative clustering

In [18]:
from sklearn.cluster import AgglomerativeClustering, KMeans

In [19]:
kmeans = KMeans(n_cluster)
cluster_labels_kmeans = kmeans.fit_predict(data.drop(class_var, axis=1))
print(NMI(data.species, cluster_labels_kmeans))

0.758205727819


In [20]:
agg = AgglomerativeClustering(n_cluster)
cluster_labels_agg = agg.fit_predict(data.drop(class_var, axis=1))
print(NMI(data.species, cluster_labels_agg))

0.770140990573


#### Try with scaled input

In [21]:
scaled_data = data.drop(class_var, axis=1)
scaled_data = (scaled_data - scaled_data.mean(axis=0)) / scaled_data.std()

In [22]:
cluster_labels_kmeans_scaled = kmeans.fit_predict(scaled_data)
print(NMI(data.species, cluster_labels_kmeans_scaled))

0.652558221252


In [23]:
cluster_labels_agg_scaled = agg.fit_predict(data.drop(class_var, axis=1))
print(NMI(data.species, cluster_labels_agg_scaled))

0.770140990573


Clustering performance of proposed method outperforms K-means clustering,

and is able to provide explanation for each cluster at the same time.